In [1]:
import pandas as pd
import numpy as np
import os
import re

Используем датасет кусочков кода с GitHub и сообщения без кода для обучения.

In [2]:
code_df = pd.read_csv(os.path.join('..', 'data', 'GITHUB_CODE.csv'))
msg_df = pd.read_csv(os.path.join('..', 'data', 'OTHER.csv'))
init_df = pd.concat((code_df, msg_df))
init_df = init_df.drop(columns=['Unnamed: 0'])
init_df

,content,type
0,// Utility.cs\n//\n\nusing System;\nusing Syst...,cs
1,\n{$options.identity.sitename} - Your Accounts...,txt
2,"{\n ""name"": ""opulence/sessions"",\n ""descript...",json
3,package io.mycat.backend.postgresql.packet;\n\...,java
4,TAPi18n._afterUILanguageChange = function() {\...,js
...,...,...
28882,Разрабатываете 😳 ? Открытый проект ? MicroSCAD...,other
28883,Можно ли еще что-либо оптимизировать при такой...,other
28884,"Попробуйте через IDE проходить курс, гораздо ...",other
28885,"В этом чате недавно было, что можно немного по...",other


Вычленним только те яыки, которые нас интересуют

In [3]:
values_to_filter = ['py', 'cpp', 'js', 'java', 'yaml', 'bash', 'sh', 'markdown', 'md', 'c', 'kt', 'hs', 'other']
df = init_df[init_df['type'].isin(values_to_filter)]
df

,content,type
3,package io.mycat.backend.postgresql.packet;\n\...,java
4,TAPi18n._afterUILanguageChange = function() {\...,js
6,/**\n * \n */\npackage org.openrtb.dsp.intf.mo...,java
12,import Promise from 'bluebird';\n\nconst ONE_S...,js
22,//>>built\ndefine(\n//begin v1.x content\n({\n...,js
...,...,...
28882,Разрабатываете 😳 ? Открытый проект ? MicroSCAD...,other
28883,Можно ли еще что-либо оптимизировать при такой...,other
28884,"Попробуйте через IDE проходить курс, гораздо ...",other
28885,"В этом чате недавно было, что можно немного по...",other


In [4]:
df.loc[df['type'] == 'py', 'type'] = 'PYTHON' 
df.loc[(df['type'] == 'sh') | (df['type'] == 'bash'), 'type'] = 'BASH'
df.loc[df['type'] == 'c', 'type'] = 'C'
df.loc[df['type'] == 'cpp', 'type'] = 'CPP'
df.loc[df['type'] == 'hs', 'type'] = 'HASKELL'
df.loc[df['type'] == 'java', 'type'] = 'JAVA'
df.loc[df['type'] == 'js', 'type'] = 'JAVASCRIPT'
df.loc[df['type'] == 'kt', 'type'] = 'KOTLIN'
df.loc[(df['type'] == 'markdown') | (df['type'] == 'md'), 'type'] = 'MARKDOWN'
df.loc[df['type'] == 'yaml', 'type'] = 'YAML'
df.loc[df['type'] == 'other', 'type'] = 'OTHER'

C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\4077853206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['type'] == 'py', 'type'] = 'PYTHON'
C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\4077853206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(df['type'] == 'sh') | (df['type'] == 'bash'), 'type'] = 'BASH'
C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\4077853206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [5]:
df

,content,type
3,package io.mycat.backend.postgresql.packet;\n\...,JAVA
4,TAPi18n._afterUILanguageChange = function() {\...,JAVASCRIPT
6,/**\n * \n */\npackage org.openrtb.dsp.intf.mo...,JAVA
12,import Promise from 'bluebird';\n\nconst ONE_S...,JAVASCRIPT
22,//>>built\ndefine(\n//begin v1.x content\n({\n...,JAVASCRIPT
...,...,...
28882,Разрабатываете 😳 ? Открытый проект ? MicroSCAD...,OTHER
28883,Можно ли еще что-либо оптимизировать при такой...,OTHER
28884,"Попробуйте через IDE проходить курс, гораздо ...",OTHER
28885,"В этом чате недавно было, что можно немного по...",OTHER


In [6]:
lang_to_int = {'BASH': 0,
               'PYTHON': 1,
               'KOTLIN': 2,
               'JAVA': 3,
               'JAVASCRIPT': 4,
               'C': 5,
               'HASKELL': 6,
               'MARKDOWN': 7,
               'YAML': 8,
               'CPP': 9,
               'OTHER': 10}
int_to_lang = {0: 'BASH',
               1: 'PYTHON',
               2: 'KOTLIN',
               3: 'JAVA',
               4: 'JAVASCRIPT',
               5: 'C',
               6: 'HASKELL',
               7: 'MARKDOWN',
               8: 'YAML',
               9: 'CPP',
               10: 'OTHER'}

df['type'] = df['type'].map(lang_to_int)
df

C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\1201140344.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'] = df['type'].map(lang_to_int)


,content,type
3,package io.mycat.backend.postgresql.packet;\n\...,3
4,TAPi18n._afterUILanguageChange = function() {\...,4
6,/**\n * \n */\npackage org.openrtb.dsp.intf.mo...,3
12,import Promise from 'bluebird';\n\nconst ONE_S...,4
22,//>>built\ndefine(\n//begin v1.x content\n({\n...,4
...,...,...
28882,Разрабатываете 😳 ? Открытый проект ? MicroSCAD...,10
28883,Можно ли еще что-либо оптимизировать при такой...,10
28884,"Попробуйте через IDE проходить курс, гораздо ...",10
28885,"В этом чате недавно было, что можно немного по...",10


In [7]:
df['content'].str.len().min(), df['content'].str.len().max() 

(22, 2529)

In [8]:
import string

def get_features_puctuation(text: str):
    features = []
    text = str(text)
    # print("get punctuation features on", text)
    for sign in string.punctuation:
        features.append(text.count(sign))
    return features

def remove_punctuation(text: str):
    return re.sub(r"[^\w\s]+", ' ', text).replace('_', ' ')

def count_uppercase_letters(text: str):
    return sum(1 for char in text if char.isupper())

def camel_case_split(text: str):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', text.replace('\n', ' '))
    return ' '.join([m.group(0) for m in matches])



def get_text_features(text):
    text = str(text)
    punct = get_features_puctuation(text)
    punct.append(count_uppercase_letters(text))
    return punct

def preprocess_text(text):
    text = str(text)
    return ' '.join(camel_case_split(remove_punctuation(text)).lower().split())

In [9]:
test_string = 'int main(int argc, char **argv) {\n std::cout << "HelloWorld!" << std::endl;\n return 0;\n }'
print(get_text_features(test_string))

[1, 2, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 0, 4, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2]


In [10]:
df['features'] = df['content'].apply(get_text_features)
df['preprocessed_text'] = df['content'].apply(preprocess_text)

C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\4024756841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['features'] = df['content'].apply(get_text_features)
C:\Users\sergf\AppData\Local\Temp\ipykernel_19996\4024756841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed_text'] = df['content'].apply(preprocess_text)


В качестве модели будем использовать логистическую регрессию, а также CountVectorizer для анализа текста. Поскольку присутствует явный дизбаланс классов (сниппетов питона больше 17 тысяч, а Хаскеля всего 8), будем использовать weighted f1_score в качестве метрики.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Выделим часть датасета с GitHub под тест, потому что это единственный способ наверняка понять, как работает модель (разметка данных из ТГ очень плохая).

In [12]:
train, test = train_test_split(df, random_state=0, test_size=0.1)

In [13]:
from scipy.sparse import csr_matrix, hstack

In [14]:
bow = CountVectorizer()
bow_features_train = bow.fit_transform(train['preprocessed_text'])
text_features_train = csr_matrix(np.vstack(train['features']))
x_train = hstack((bow_features_train, text_features_train))
y_train = train['type']

bow_features_test = bow.transform(test['preprocessed_text'])
text_features_test = csr_matrix(np.vstack(test['features']))
x_test = hstack((bow_features_test, text_features_test))
y_test = test['type']

In [15]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

c:\Users\sergf\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [16]:
y_pred = model.predict(x_test)
f1_score(y_pred, y_test, average='weighted')

0.9764559701023453

In [17]:
def make_test_from_tg(bow: CountVectorizer, _df: pd.DataFrame):
    df = _df.copy()
    df['features'] = df['snippets'].apply(get_text_features)
    df['preprocessed_text'] = df['snippets'].apply(preprocess_text)
    bow_features_test = bow.transform(df['preprocessed_text'])
    text_features_test = csr_matrix(np.vstack(df['features']))
    x_test = hstack((bow_features_test, text_features_test))
    y_test = df['language']
    return x_test, y_test

In [18]:
dfs = []
DATA_PATH = os.path.join('..', 'data', 'code_snippets')

for filename in os.listdir(DATA_PATH):
    tmp_df = pd.read_csv(os.path.join(DATA_PATH, filename))
    tmp_df['language'] = lang_to_int['_'.join(filename.split('_')[:-1])]
    dfs.append(tmp_df)

test_df = pd.concat(dfs)
test_df

,Unnamed: 0,snippets,language
0,0,root@pve:/# lvs | grep vm-109\n vm-109-disk-0...,0
1,1,"exec(""sudo -u bratsk /usr/bin/sshpass -p admin...",0
2,2,"$bar1 = <<<EOT\n""\""""bitmain-freq1""\"""" : ""\""""64...",0
3,3,"""bitmain-freq1"" : ""625"",\n""bitmain-freq2"" : ""6...",0
4,4,script_sql && \\nscript_php && \\nscript_jq & ...,0
...,...,...,...
17231,17231,"\nasync def test_get_notifications(bearer: MM,...",1
17232,17232,/*\n\n*/,1
17233,17233,In [1]: print('hello'); print('world'); print(...,1
17234,17234,"Authorization: `Bearer ${token}`,",1


In [19]:
test_df.dropna(axis=1, how='any')
x_test_tg, y_test_tg = make_test_from_tg(bow, test_df)
y_pred_tg = model.predict(x_test_tg)

Поскольку найти чатики отдельно по С и С++ не представлялось возможным, я добавил только сниппеты с пометкой С, но если модель угадала их как С++, я тоже буду это считать.

In [20]:
y_test_tg = list(y_test_tg)
y_pred_tg = list(y_pred_tg)
for i, pred in enumerate(y_pred_tg):
    if int_to_lang[y_test_tg[i]] == 'C' and int_to_lang[pred] in ['C', 'CPP']:
        y_pred_tg[i] = lang_to_int['C']

In [21]:
f1_score(y_pred_tg, y_test_tg, average='weighted')

0.3779924600957938

Результат получился не слишком высокий, но стоит иметь в виду, что данные для тестирования максимально плохие. В чатах далеко не всегда появляются сообщения с кодом языка, которому этот чат посвящён, а зачастую там вообще просто текст или вывод из консоли. Поэтому, по субъективным ощущениям, результат довольно неплохой, что показали тесты на собственных сниппетах.

Для тестов брались сниппеты из нескольких реальных сообщений, а также из моих проектов. Для большей реалистичности, в тестовых сниппетах нарочно были добавлены ошибки и местами испорчен синтаксис, но модель всё равно хорошо справилась.

In [22]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump((model, bow), f)